In [1]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

In [3]:
openasl = pd.read_csv('openasl-v1.0.tsv', sep='\t')
openasl.head()

/tmp/ipykernel_14040/844222082.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  openasl = pd.read_csv('openasl-v1.0.tsv', sep='\t')


,vid,yid,start,end,raw-text,tokenized-text,gloss,split
0,Mci9oyb5V2E-00:00:06.000-00:00:06.589,Mci9oyb5V2E,00:00:06.000,00:00:06.589,Hello!,hello,NaN,train
1,RjJSxzMkC90-00:00:38.905-00:00:39.975,RjJSxzMkC90,00:00:38.905,00:00:39.975,Then it was time.,then it was time,NaN,train
2,RjJSxzMkC90-00:00:37.970-00:00:38.905,RjJSxzMkC90,00:00:37.970,00:00:38.905,"I said, ""oh.""",i said oh,NaN,train
3,RjJSxzMkC90-00:00:25.123-00:00:27.524,RjJSxzMkC90,00:00:25.123,00:00:27.524,Bush through relay.,bush through relay,NaN,train
4,RjJSxzMkC90-00:00:14.083-00:00:17.094,RjJSxzMkC90,00:00:14.083,00:00:17.094,The secretary called me to come.,the secretary called me to come,NaN,train


Original Split between train, dev and test

In [4]:
openasl['split'].value_counts(normalize=True) * 100

split
train    98.025808
test      0.991668
valid     0.982524
Name: proportion, dtype: float64

run facial recognition on a sample video

frame extraction

In [5]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [2]:
import cv2
import numpy as np
from pathlib import Path

In [7]:
videoname = input()
cap = cv2.VideoCapture(videoname)
total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
indices = np.linspace(200, total - 1, 20).astype(int)

In [8]:
saveto = input()
for idx in indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, int(idx))
        ok, frame = cap.read()
        if not ok:
            continue

        frame_name = f"{videoname}_{idx}.jpg"
        cv2.imwrite(str(Path(saveto) / frame_name), frame)

Load LVFace

In [9]:
from inference_onnx import LVFaceONNXInferencer

In [15]:
## Initialize the inferencer from inference_onnx.py
inferencer = LVFaceONNXInferencer(
   model_path="LVFace-B_Glint360K.onnx",  # Path to your ONNX model
   use_gpu=False # Set to False for CPU-only 
)

def get_embedding(saveto, videoname):
      emblist = []
      for frame in Path(saveto).glob(videoname + "*.jpg"):
            emb = inferencer.infer_from_image(str(frame))
            emb = emb.squeeze()
            emblist.append((emb))
      return emblist

def get_average(emb):
      emb = np.stack(emb, axis = 0)
      avg = np.mean(emb, axis = 0)
      avg = avg / np.linalg.norm(avg)
      return avg

avg_emb = [get_average(get_embedding("frames1", "videoexample.mp4")), get_average(get_embedding("frames2", "videoexample2.mp4"))]

avg_embs = np.stack(avg_emb, axis=0)


simmatrix = np.dot(avg_embs,avg_embs.T)

simmatrix

array([[0.99999994, 0.24117297],
       [0.24117297, 0.99999976]], dtype=float32)

checkout after doing the whole dataset with gpu

In [12]:
import numpy as np
from inference_onnx import LVFaceONNXInferencer
import pandas as pd

In [8]:
embs = np.load('video_embeddings.npy')
inferencer = LVFaceONNXInferencer(
   model_path="LVFace-L_Glint360K.onnx",  # Path to your ONNX model
   use_gpu=False # Set to False for CPU-only 
)
inferencer.calculate_similarity(embs[5], embs[22])

np.float32(0.38754505)

In [13]:
pd.DataFrame(embs)

,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
0,0.046632,-0.001189,0.027962,-0.055570,-0.003337,-0.005630,-0.035967,0.005482,-0.015888,0.009669,...,0.013585,-0.002291,0.056957,0.047845,0.003731,-0.060871,-0.065511,0.010138,-0.012471,-0.025235
1,0.038842,0.023890,0.014728,-0.060894,-0.011833,-0.014557,-0.032060,-0.015496,0.001511,0.072228,...,0.051549,0.087488,0.029862,0.073605,-0.057583,-0.007119,-0.082305,0.015020,0.013523,0.001694
2,0.103519,0.010489,-0.013828,-0.077661,0.046248,-0.016462,0.009756,0.080561,-0.022555,-0.021059,...,0.048815,0.049904,0.038544,0.047981,-0.015301,0.004189,-0.044899,0.010137,0.008784,-0.044250
3,0.064417,0.041915,0.031755,-0.037319,0.037660,-0.019348,-0.002220,-0.011491,0.029452,0.038583,...,-0.004345,0.047395,0.079437,0.059383,-0.032424,0.008416,-0.078313,0.027165,-0.032241,-0.067763
4,0.053487,-0.023552,0.010458,0.006519,0.091084,0.000292,-0.033966,0.019700,0.030498,0.044415,...,0.033306,0.071579,0.080527,0.072907,-0.024368,-0.040059,-0.095629,0.053498,0.023647,0.006279
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
775,0.054898,-0.007323,-0.007001,-0.010465,0.074892,-0.000480,-0.010423,0.028825,0.019759,-0.005353,...,0.018118,0.071754,0.087436,0.079310,-0.008344,-0.065189,-0.059824,0.059722,0.000213,-0.012727
776,0.064189,-0.005378,0.010271,-0.066538,0.014958,-0.001770,-0.047323,0.029855,0.003354,0.007133,...,0.023776,-0.017494,0.070100,0.032517,0.005955,-0.081312,-0.058422,0.030354,0.001205,-0.040967
777,0.070164,0.023249,0.016316,-0.038726,0.012763,0.005376,-0.020860,-0.054237,0.017861,0.041762,...,0.019351,0.048662,0.065259,0.040158,-0.047420,-0.013353,-0.096841,0.016069,-0.000266,0.013263
778,0.039145,0.079176,-0.001519,-0.039856,0.093941,-0.082101,0.010924,-0.051287,0.008428,-0.062057,...,-0.048465,0.019720,0.004274,0.092479,-0.046647,0.022139,-0.130263,-0.057514,-0.008669,0.043136


In [15]:
feat1 = inferencer.infer_from_image('img1.png')
feat2 = inferencer.infer_from_image('img2.png')

In [17]:
inferencer.calculate_similarity(feat1, feat2)

np.float32(0.39734736)

STEM Wiki

In [ ]:
stem = pd.read_csv('all.csv')
stem.head()

,user,articleName,sectionIndex,sentenceIndex,filename,sentence,fs_span,fs_text
0,62ab763c2653e8005020f166,Acid catalysis,0,0,01655119426829943-2053312_0_0.mp4,"In acid catalysis and base catalysis, a chemic...","[(80, 100), (115, 173), (227, 260), (409, 432)...","['acid', 'catalysis', 'base', 'acid', 'base']"
1,62ab763c2653e8005020f166,Acid catalysis,0,1,5813571323095919-2053312_0_1.mp4,"By Brønsted–Lowry acid–base theory, the acid i...","[(64, 199), (224, 295), (336, 356)]","['Brønsted–Lowry', 'acid–base', 'acid']"
2,62ab763c2653e8005020f166,Acid catalysis,0,2,8261011694304976-2053312_0_2.mp4,Typical reactions catalyzed by proton transfer...,"[(220, 285), (316, 362)]","['esterfications', 'aldol']"
3,62ab763c2653e8005020f166,Acid catalysis,0,3,023397513349495913-2053312_0_3.mp4,"In these reactions, the conjugate acid of the ...","[(86, 100), (105, 186), (198, 215), (259, 336)...","['the', 'conjugate', 'acid', 'carbonyl', 'elec..."
4,62ab763c2653e8005020f166,Acid catalysis,0,4,2738300105048981-2053312_0_4.mp4,Depending on the chemical species that act as ...,"[(131, 144), (169, 189), (204, 218), (243, 317...","['act', 'acid', 'be', 'catalytic', 'catalysis'..."


count signers

In [ ]:
stem['user'].value_counts()

user
62ab763c2653e8005020f166    38
62a0cece9bbab800502f2e90    38
62a25e632e0bec00503747ad    38
62ab8ca14eede20050bef000    36
62a20629ff311c0050d35493    35
6231157f9e3b770048c76dd4    34
629f640e0a2f100050eeaa6a    34
62ac8d364eede20050bef369    33
62a6a8a1cba00d00501964df    33
62a1edef6110330050235169    31
624b3778f1a4ec0048766482    31
62acd9b4b7e7010050c6bb5e    31
62a90031c0bfef0050fdbef7    28
62a27b0dff311c0050d3572e    27
62a0d78a9bbab800502f2eb5    26
Name: count, dtype: int64